In [90]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import healpy as hp
from astropy.io import fits

In [91]:
# Frequencies of Planck
HF = [100, 143, 217, 353, 545, 857] # [GHz]

In [92]:
# Get smoot values from PSF
R = fits.getdata("../Maps/Initial/HFI_RIMO_R2.00.fits", 2)
FWHM = R[:]["FWHM"]

In [93]:
# Create DataFrame with frequencies and FWHM
data = pd.DataFrame(data=[HF,FWHM]).T
data.columns = ["Frequency", "FWHM"]

In [94]:
# Getting the PSF map for each frequency
data["PSF Map"] = data["Frequency"].map(
    lambda x: hp.read_map(f"../Maps/Initial/HFI_SkyMap_{int(x)}_2048_R2.00_full.fits")
)

In [95]:
data["Mod FWHM"] = data["FWHM"].map(
    lambda x: np.sqrt(100-pow(x,2))
)

In [96]:
data

,Frequency,FWHM,PSF Map,Mod FWHM
0,100.0,9.682,"[-0.00012238526, -9.958545e-05, -3.9736184e-05...",2.501775
1,143.0,7.303,"[-9.868725e-05, -7.245516e-05, -7.861525e-05, ...",6.831266
2,217.0,5.021,"[-5.005044e-05, -2.8069558e-06, 1.2417856e-05,...",8.648096
3,353.0,4.944,"[0.00039311356, 0.00031349497, 0.00053247873, ...",8.692345
4,545.0,4.831,"[0.4312183, 0.4332836, 0.4331446, 0.48448372, ...",8.755652
5,857.0,4.638,"[0.890403, 0.8909181, 0.89945364, 0.87775695, ...",8.859399


In [ ]:
# Smoothing [rad] in [10 arcmin] for each map
data["Smooth Map"] = data.apply(
    lambda x: hp.smoothing( x["PSF Map"], fwhm = x["Mod FWHM"]),
    axis = 1
)

In [ ]:
# Degrade de pixel resolution from 2048 to 512
data["Degraded Map"] = data["Smooth Map"].map(
    lambda x: hp.ud_grade( x, 512)
)

In [ ]:
# Change units for frequency maps 545 and 857 to K_CMB
data["K_CMB Map"] = data["Degraded Map"]
data["K_CMB Map"][data["Frequency"] == 545] = data["Degraded Map"][data["Frequency"] == 545]/58.04
data["K_CMB Map"][data["Frequency"] == 857] = data["Degraded Map"][data["Frequency"] == 857]/2.27


In [ ]:
# Write each frequency map into a healpix FITS file.
data["K_CMB Map"].apply(
    lambda x: hp.write_map(
        f"../Maps/Generated/HFI_{x["Frequency"]}.fits", x["K_CMB Map"]),
    axis = 1
)

In [ ]:
hp.mollview(data["Frequency" == 545]["K_CMB Map"])